In [5]:
'''
PRAW ONLY GIVES ACCESS TO MOST RECENT POSTS
import praw
from dotenv import load_dotenv
import os

load_dotenv()

client_id = os.getenv("REDDIT_CLIENT_ID")
client_secret = os.getenv("REDDIT_SECRET")
user_agent = os.getenv("REDDIT_USER_AGENT")


reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent
)

for submission in reddit.subreddit("wallstreetbets").new(limit=1000):
    print(submission.title)
'''

1749438760.0
1749433176.0
1749430657.0
1749419134.0
1749412653.0
1749401707.0
1749393282.0
1749392799.0
1749390978.0
1749389415.0
1749370184.0
1749351843.0
1749340537.0
1749324752.0
1749314534.0
1749312656.0
1749308506.0
1749306524.0
1749304937.0
1749297993.0
1749280913.0
1749280827.0
1749259793.0
1749253134.0
1749251692.0
1749249731.0
1749249644.0
1749249002.0
1749248521.0
1749248419.0
1749242156.0
1749242121.0
1749240087.0
1749239836.0
1749238891.0
1749238035.0
1749237038.0
1749236028.0
1749235594.0
1749234207.0
1749234192.0
1749231583.0
1749230421.0
1749228986.0
1749227898.0
1749227836.0
1749226542.0
1749226097.0
1749225215.0
1749223826.0
1749223230.0
1749222911.0
1749222575.0
1749222144.0
1749222100.0
1749221846.0
1749221474.0
1749221214.0
1749219765.0
1749218662.0
1749218326.0
1749218229.0
1749217916.0
1749217827.0
1749217717.0
1749213387.0
1749203860.0
1749190939.0
1749187369.0
1749181944.0
1749178714.0
1749176710.0
1749175688.0
1749168566.0
1749167469.0
1749167468.0
1749167023.0

In [14]:
import kagglehub
import os

os.environ['KAGGLEHUB_CACHE'] = os.getcwd() + '\\data'

path = kagglehub.dataset_download("gpreda/reddit-wallstreetsbets-posts")

print("Path to dataset files:", path)

100%|██████████| 16.5M/16.5M [00:00<00:00, 29.3MB/s]

Extracting files...


Path to dataset files: C:\Users\jerry\PycharmProjects\StockPredictor\data\datasets\gpreda\reddit-wallstreetsbets-posts\versions\106


In [1]:
import pandas as pd
wsb_posts = pd.read_csv('data\\datasets\\gpreda\\reddit-wallstreetsbets-posts\\versions\\106\\reddit_wsb.csv')
wsb_posts

,title,score,id,url,comms_num,created,body,timestamp
0,"It's not about the money, it's about sending a...",55,l6ulcx,https://v.redd.it/6j75regs72e61,6,1.611863e+09,NaN,2021-01-28 21:37:41
1,Math Professor Scott Steiner says the numbers ...,110,l6uibd,https://v.redd.it/ah50lyny62e61,23,1.611862e+09,NaN,2021-01-28 21:32:10
2,Exit the system,0,l6uhhn,https://www.reddit.com/r/wallstreetbets/commen...,47,1.611862e+09,The CEO of NASDAQ pushed to halt trading “to g...,2021-01-28 21:30:35
3,NEW SEC FILING FOR GME! CAN SOMEONE LESS RETAR...,29,l6ugk6,https://sec.report/Document/0001193125-21-019848/,74,1.611862e+09,NaN,2021-01-28 21:28:57
4,"Not to distract from GME, just thought our AMC...",71,l6ufgy,https://i.redd.it/4h2sukb662e61.jpg,156,1.611862e+09,NaN,2021-01-28 21:26:56
...,...,...,...,...,...,...,...,...
53182,What I Learned Investigating SAVA FUD Spreaders,238,owd2pn,https://www.reddit.com/r/wallstreetbets/commen...,87,1.627906e+09,***TLDR: Three bitter scientists partnered up ...,2021-08-02 15:03:27
53183,"Daily Popular Tickers Thread for August 02, 20...",228,owd1a5,https://www.reddit.com/r/wallstreetbets/commen...,1070,1.627906e+09,\nYour daily hype thread. Please keep the shit...,2021-08-02 15:01:03
53184,Hitler reacts to the market being irrational,7398,owc5dr,https://v.redd.it/46jxu074exe71,372,1.627902e+09,NaN,2021-08-02 13:59:35
53185,"Daily Discussion Thread for August 02, 2021",338,owbfjf,https://www.reddit.com/r/wallstreetbets/commen...,11688,1.627898e+09,Your daily trading discussion thread. Please k...,2021-08-02 13:00:16


In [8]:
from transformers import pipeline

sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english",
    device=0
)

batch_size = 2048

def batch_sentiment_analysis(texts, batch_size=batch_size):
    results = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        batch_results = sentiment_analyzer(batch)
        results.extend(batch_results)
    return results

sentiments = batch_sentiment_analysis(wsb_posts['title'].tolist(), batch_size=batch_size)

wsb_posts['sentiment'] = [res['label'] for res in sentiments]
wsb_posts['score'] = [res['score'] for res in sentiments]

wsb_posts


Device set to use cuda:0


,title,score,id,url,comms_num,created,body,timestamp,sentiment
0,"It's not about the money, it's about sending a...",0.993992,l6ulcx,https://v.redd.it/6j75regs72e61,6,1.611863e+09,NaN,2021-01-28 21:37:41,NEGATIVE
1,Math Professor Scott Steiner says the numbers ...,0.999742,l6uibd,https://v.redd.it/ah50lyny62e61,23,1.611862e+09,NaN,2021-01-28 21:32:10,NEGATIVE
2,Exit the system,0.999687,l6uhhn,https://www.reddit.com/r/wallstreetbets/commen...,47,1.611862e+09,The CEO of NASDAQ pushed to halt trading “to g...,2021-01-28 21:30:35,NEGATIVE
3,NEW SEC FILING FOR GME! CAN SOMEONE LESS RETAR...,0.996106,l6ugk6,https://sec.report/Document/0001193125-21-019848/,74,1.611862e+09,NaN,2021-01-28 21:28:57,NEGATIVE
4,"Not to distract from GME, just thought our AMC...",0.996898,l6ufgy,https://i.redd.it/4h2sukb662e61.jpg,156,1.611862e+09,NaN,2021-01-28 21:26:56,NEGATIVE
...,...,...,...,...,...,...,...,...,...
53182,What I Learned Investigating SAVA FUD Spreaders,0.990813,owd2pn,https://www.reddit.com/r/wallstreetbets/commen...,87,1.627906e+09,***TLDR: Three bitter scientists partnered up ...,2021-08-02 15:03:27,NEGATIVE
53183,"Daily Popular Tickers Thread for August 02, 20...",0.944735,owd1a5,https://www.reddit.com/r/wallstreetbets/commen...,1070,1.627906e+09,\nYour daily hype thread. Please keep the shit...,2021-08-02 15:01:03,NEGATIVE
53184,Hitler reacts to the market being irrational,0.993521,owc5dr,https://v.redd.it/46jxu074exe71,372,1.627902e+09,NaN,2021-08-02 13:59:35,NEGATIVE
53185,"Daily Discussion Thread for August 02, 2021",0.817849,owbfjf,https://www.reddit.com/r/wallstreetbets/commen...,11688,1.627898e+09,Your daily trading discussion thread. Please k...,2021-08-02 13:00:16,POSITIVE


In [9]:
wsb_posts.to_csv('data\\wsb_posts_with_sentiment.csv')